<a href="https://colab.research.google.com/github/shazzad-hasan/practice-deep-learning-with-pytorch/blob/main/text_classification/tweet_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# upload kaggle API key from your local machine
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shazzadraihan","key":"da63bbe0f8dcb3bd7fb35034046ca758"}'}

In [2]:
# make a kaggle dir, copy the API key to it
# and make sure the file in only readable by yourself (chmod 600)
!mkdir ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# use API command to download the dataset
!kaggle datasets download -d kazanova/sentiment140

 85% 69.0M/80.9M [00:00<00:00, 115MB/s]
100% 80.9M/80.9M [00:00<00:00, 96.0MB/s]


In [4]:
# uncompress the dataset
!unzip -qq sentiment140.zip

In [5]:
!pip install torchtext==0.9.1
!pip install torch==1.8.1
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.1 MB 1.3 MB/s 
     |████████████████████████████████| 804.1 MB 2.5 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whee

In [6]:
# import required libraries
import torch
import torchtext
from torchtext.legacy import data

import numpy as np
import pandas as pd
import random
import spacy

In [7]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
  print("CUDA is not available")
else:
  print("CUDA is available")

device = torch.device('cuda') if train_on_gpu else torch.device('cpu')

CUDA is available


In [8]:
tweets_df = pd.read_csv("training.1600000.processed.noemoticon.csv",
                       encoding='latin-1', header=None)

In [9]:
tweets_df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
tweets_df[0].value_counts()

0    800000
4    800000
Name: 0, dtype: int64

### Pre-process the dataset

In [11]:
# create a column of type category from the label column
tweets_df["sentiment_cat"] = tweets_df[0].astype('category')
# encode category column as numerical info in another column (sentiment)
tweets_df["sentiment"] = tweets_df["sentiment_cat"].cat.codes
# save the modified csv back to dist
tweets_df.to_csv("train-processed.csv", header=None, index=None)   
# sample come tweets for testing and save it as another csv   
tweets_df.sample(10000).to_csv("train-processed-sample.csv", header=None, index=None)

In [12]:
LABEL = data.LabelField() 
TWEET = data.Field('spacy', tokenizer_language='en_core_web_sm', lower=True)

# carete a list that maps the fields onto the list of rows that are in the tweets dataframe
fields = [('score',None), ('id',None), ('date',None), ('query',None),
          ('name',None), ('tweet', TWEET), ('category',None), ('label',LABEL)]

In [13]:
tweet_data = data.dataset.TabularDataset(
        path="train-processed-sample.csv", 
        format="CSV", 
        fields=fields,
        skip_header=False)

In [14]:
# split the dataset into train, test, and validation sets
(train_data, test_data, valid_data) = tweet_data.split(split_ratio=[0.6,0.2,0.2],
                                            stratified=True, strata_field='label')

print("Num of training data: ", len(train_data))
print("Num of test data: ", len(test_data))
print("Num of validation_data: ", len(valid_data))

Num of training data:  6000
Num of test data:  2000
Num of validation_data:  2000


In [15]:
# limit the vocabulary in the training data
vocab_size = 20000
TWEET.build_vocab(train_data, max_size = vocab_size)
LABEL.build_vocab(train_data)
# most common words in the vocabulary
TWEET.vocab.freqs.most_common(10)

[('i', 2743),
 ('to', 2129),
 ('the', 1975),
 ('a', 1428),
 ('my', 1182),
 ('and', 1152),
 ('you', 882),
 ('is', 856),
 ('in', 799),
 ('for', 766)]

In [16]:
# define data loader
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = 32,
    device = device,
    sort_key = lambda x: len(x.tweet),
    sort_within_batch = False)

### Define a model

In [17]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(Net, self).__init__()
    
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size=embedding_dim,  
                hidden_size=hidden_size, num_layers=1)
        self.predictor = nn.Linear(hidden_size, 2)

    def forward(self, seq):
        output, (hidden,_) = self.encoder(self.embedding(seq))
        preds = self.predictor(hidden.squeeze(0))
        return preds

model = Net(100,300, 20002)
model.to(device)

Net(
  (embedding): Embedding(20002, 300)
  (encoder): LSTM(300, 100)
  (predictor): Linear(in_features=100, out_features=2, bias=True)
)

### Define a loss function and an optimizer

In [18]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()

### Train the model

In [19]:
def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs+1):
     
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * batch.tweet.size(0)
        training_loss /= len(train_iterator)
 
        
        model.eval()
        for batch_idx,batch in enumerate(valid_iterator):
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            valid_loss += loss.data.item() * batch.tweet.size(0)
 
        valid_loss /= len(valid_iterator)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))

In [20]:
train(5, model, optimizer, criterion, train_iterator, valid_iterator)        

Epoch: 1, Training Loss: 19.62, Validation Loss: 10.78
Epoch: 2, Training Loss: 17.99, Validation Loss: 11.73
Epoch: 3, Training Loss: 15.29, Validation Loss: 12.49
Epoch: 4, Training Loss: 13.48, Validation Loss: 12.64
Epoch: 5, Training Loss: 12.06, Validation Loss: 13.15


### Test the model

In [21]:
def classify_tweet(tweet):
    categories = {0: "Negative", 1:"Positive"}
    processed = TWEET.process([TWEET.preprocess(tweet)])
    processed = processed.to(device)
    model.eval()
    return categories[model(processed).argmax().item()]

In [22]:
def random_deletion(words, p=0.5):
    if len(words) == 1:
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    if len(remaining) == 0:
        return [random.choice(words)]
    else:
        return remaining

In [23]:
def random_swap(sentence, n=5):
    length = range(len(sentence))
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1]
    return sentence

In [24]:
# Note: you'll have to define remove_stopwords() and get_synonyms() elsewhere

def random_insertion(sentence,n):
    words = remove_stopwords(sentence)
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym)
    return sentence

In [25]:
import googletrans

translator = googletrans.Translator()

sentences = ['The cat sat on the mat']

translations_fr = translator.translate(sentences, dest='fr')
fr_text = [t.text for t in translations_fr] 
translations_en = translator.translate(fr_text, dest='en')
en_text = [t.text for t in translations_en]
print(en_text)   

available_langs = list(googletrans.LANGUAGES.keys())
tr_lang = random.choice(available_langs)
print(f"Translating to {googletrans.LANGUAGES[tr_lang]}")

translations = translator.translate(sentences, dest=tr_lang)
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=tr_lang, dest='en')
en_text = [t.text for t in translations_en_random]
print(en_text)

['The cat sat on the carpet']
Translating to danish
['Katten sad på måtten']
['The cat sat on the mat']
